In [1]:
%pylab inline

import os
import numpy as np
import pandas as pd
# from scipy.misc import imread
from sklearn.metrics import accuracy_score
import tensorflow as tf

Populating the interactive namespace from numpy and matplotlib


d:\dilip\myexperiments\python\tensorflowvenv\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
d:\dilip\myexperiments\python\tensorflowvenv\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
d:\dilip\myexperiments\python\tensorflowvenv\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
d:\dilip\myexperiments\python\tensorflowvenv\l

In [19]:
# To stop potential randomness
seed = 128
rng = np.random.RandomState(seed)

In [10]:
root_dir = os.path.abspath('..')
data_dir = os.path.join(root_dir, 'data')
sub_dir = os.path.join(root_dir, 'sub')

# check for existence
os.path.exists(root_dir)
os.path.exists(data_dir)
os.path.exists(sub_dir)

True

In [9]:
print(sub_dir)

D:\Dilip\MyExperiments\Python\tensorflowvenv\sub


In [16]:
train = pd.read_csv(os.path.join(data_dir, 'Train', 'train.csv'))
test = pd.read_csv(os.path.join(data_dir, 'Test','Test.csv'))
sample_submission = pd.read_csv(os.path.join(data_dir, 'Sample_Submission.csv'))


In [21]:
sample_submission.head()

,ImageId,Label
0,1,0
1,2,0
2,3,0
3,4,0
4,5,0


In [22]:
import struct
import numpy as np

def read_idx(filename):
    with open(filename, 'rb') as f:
        zero, data_type, dims = struct.unpack('>HBB', f.read(4))
        shape = tuple(struct.unpack('>I', f.read(4))[0] for d in range(dims))
        return np.frombuffer(f.read(), dtype=np.uint8).reshape(shape)

In [23]:
mnist_test_labels = read_idx(r'D:\Dilip\MyExperiments\Python\tensorflowvenv\MNIST\t10k-labels.idx1-ubyte')
mnist_test_images = read_idx(r'D:\Dilip\MyExperiments\Python\tensorflowvenv\MNIST\t10k-images.idx3-ubyte')
mnist_train_labels = read_idx(r'D:\Dilip\MyExperiments\Python\tensorflowvenv\MNIST\train-labels.idx1-ubyte')
mnist_train_images = read_idx(r'D:\Dilip\MyExperiments\Python\tensorflowvenv\MNIST\train-images.idx3-ubyte')


In [41]:
split_size = int(mnist_train_images.shape[0]*0.7)


SyntaxError: invalid syntax (<ipython-input-41-8326663e8ac8>, line 2)

In [29]:
train_x, val_x = mnist_train_images[:split_size], mnist_train_images[split_size:]
train_y, val_y = mnist_train_labels[:split_size], mnist_train_labels[split_size:]


In [30]:
def dense_to_one_hot(labels_dense, num_classes=10):
    """Convert class labels from scalars to one-hot vectors"""
    num_labels = labels_dense.shape[0]
    index_offset = np.arange(num_labels) * num_classes
    labels_one_hot = np.zeros((num_labels, num_classes))
    labels_one_hot.flat[index_offset + labels_dense.ravel()] = 1
    
    return labels_one_hot

def preproc(unclean_batch_x):
    """Convert values to range 0-1"""
    temp_batch = unclean_batch_x / unclean_batch_x.max()
    
    return temp_batch

def batch_creator(batch_size, dataset_length, dataset_name):
    """Create batch with random samples and return appropriate format"""
    batch_mask = rng.choice(dataset_length, batch_size)
    
    batch_x = eval(dataset_name + '_x')[[batch_mask]].reshape(-1, input_num_units)
    batch_x = preproc(batch_x)
    
    if dataset_name == 'train':
        batch_y = eval(dataset_name).ix[batch_mask, 'label'].values
        batch_y = dense_to_one_hot(batch_y)
        
    return batch_x, batch_y

In [31]:
### set all variables

# number of neurons in each layer
input_num_units = 28*28
hidden_num_units = 500
output_num_units = 10

# define placeholders
x = tf.placeholder(tf.float32, [None, input_num_units])
y = tf.placeholder(tf.float32, [None, output_num_units])

# set remaining variables
epochs = 5
batch_size = 128
learning_rate = 0.01

### define weights and biases of the neural network (refer this article if you don't understand the terminologies)

weights = {
    'hidden': tf.Variable(tf.random_normal([input_num_units, hidden_num_units], seed=seed)),
    'output': tf.Variable(tf.random_normal([hidden_num_units, output_num_units], seed=seed))
}

biases = {
    'hidden': tf.Variable(tf.random_normal([hidden_num_units], seed=seed)),
    'output': tf.Variable(tf.random_normal([output_num_units], seed=seed))
}

In [37]:
hidden_layer = tf.add(tf.matmul(x, weights['hidden']), biases['hidden'])
hidden_layer = tf.nn.relu(hidden_layer)

output_layer = tf.matmul(hidden_layer, weights['output']) + biases['output']

In [39]:
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=output_layer, logits=y))


In [40]:
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)


ValueError: No gradients provided for any variable, check your graph for ops that do not support gradients, between variables ["<tf.Variable 'Variable:0' shape=(784, 500) dtype=float32_ref>", "<tf.Variable 'Variable_1:0' shape=(500, 10) dtype=float32_ref>", "<tf.Variable 'Variable_2:0' shape=(500,) dtype=float32_ref>", "<tf.Variable 'Variable_3:0' shape=(10,) dtype=float32_ref>"] and loss Tensor("Mean:0", shape=(), dtype=float32).